In [1]:
%load_ext autoreload
%autoreload 2
import nest_asyncio
nest_asyncio.apply()
    
from lmdeploy import TurbomindEngineConfig, pipeline
from lmdeploy.vl import load_image
model_name_or_path = "/home/yerong2/models/internlm-xcomposer2-vl-7b"
# backend_config = TurbomindEngineConfig(cache_max_entry_count=0.2)
pipe = pipeline(model_name_or_path,device_map="auto")
image = load_image('tiger.jpeg')
response = pipe(('describe this image', image))
print(response.text)

/home/yerong2/local/miniconda3/envs/mllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.


Set max length to 4096


You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.


[WARNING] gemm_config.in is not found; using default GEMM algo
The image depicts a majestic tiger, its fur a striking pattern of orange and black stripes, lying comfortably on a lush green lawn. The tiger's gaze is directed towards the camera, giving the viewer a sense of connection with this magnificent creature. The tiger's relaxed posture suggests a moment of tranquility in its natural habitat. The background is blurred, drawing focus to the tiger and emphasizing its presence in the scene. The image captures the beauty and power of the tiger in a serene setting.
